In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from core.SimpleNet import SimpleNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from core.GraphDataBlock import GraphDataBlock
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import evaluate_viz_metrics
from util.network_utils import get_net_projection
from util.graph_utils import get_shortest_path_matrix, neighbor_sampling

cuda not available
cuda not available


In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
if torch.cuda.is_available():
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    print('cuda available')
    device = 'cuda'
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    print('cuda not available')
    device = 'cpu'

cuda not available


In [6]:
dataset_name = 'cora_test'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
dataset.summarise()

Data blocks of length:  [2708]
Time to create all data (s) = 0.0355
Name of dataset = cora_test
Input dimension = 1433
Number of training samples = 2708
Training labels = True


In [7]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = dataset.input_dim # input dimension
net_parameters['L'] = 2 # number of hidden layers

In [8]:
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)

Data blocks of length:  [2708]
Time to create all data (s) = 0.0141


In [9]:
# Graph net
net_parameters['H'] = 128 # number of hidden units

net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/cora_full_2/'
filename = root + 'graph_net_800.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [10]:
y_pred_1 = get_net_projection(net_1, dataset)

Data blocks of length:  [2708]
Time to create all data (s) = 0.0151


In [11]:
plot_graph_embedding(y_pred_1, dataset.labels, dataset.adj_matrix, line_alpha=0.1)

In [12]:
y_pred = get_net_projection(net_1, dataset, n_batches=10, n_components=2)

  0%|          | 0/10 [00:00<?, ?it/s]

Data blocks of length:  [271, 271, 271, 271, 271, 271, 271, 271, 270, 270]
Time to create all data (s) = 0.0364


100%|██████████| 10/10 [00:05<00:00,  2.16it/s]


In [13]:
plot_graph_embedding(y_pred, dataset.labels, dataset.adj_matrix, line_alpha=0.1)

In [14]:
np.testing.assert_array_almost_equal(y_pred_1, y_pred, decimal=5)